This notebook shows how to use icliniq_data_crawler module to collect data from iCliniq website.

In [ ]:
import icliniq_data_crawler 

In [ ]:
# Collected data will be saved into this file.
file_name = "icliniq_test.csv"

# Start collecting data from this page number.
starting_page = 0

In [ ]:
icliniq_data_crawler.start_collecting_data(file_name, starting_page)